In [1]:
%%bash

git pull

Already up to date.


# Kernel
python3.10で動作確認（3.12ではtensorflowがinstallできない）

In [3]:
%pip install --upgrade pip

In [7]:
%pip install gymnasium
%pip install tensorflow
%pip install tf_agents
%pip install keras
%pip install numpy
%pip install matplotlib
%pip install -e .

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Obtaining file:///C:/Users/takaf/workspace/rainforcement-learningNote: you may need to restart the kernel to use updated packages.

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  

In [8]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

import nekoengineer
import nekoengineer.rl.replay_buffer as rb
import nekoengineer.rl.model as model
import math

In [9]:
# GPUが使用可能かどうかを確認
# 0: not GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [10]:
# Specify the `render_mode` parameter to show the attempts of the agent in a pop up window.
env_name = "Acrobot-v1"#"nekoengineer/CatPlayground-v0"
env = gym.make(env_name, render_mode="human")

num_states = env.observation_space.shape[0]
print("Size of State Space ->  {}".format(num_states))
num_actions = env.action_space.n
print("Size of Action Space ->  {}".format(num_actions))

print(env.reset())

Size of State Space ->  6
Size of Action Space ->  3
(array([ 0.9999337 , -0.01151325,  0.99986345,  0.01652431,  0.03277626,
        0.08876526], dtype=float32), {})


In [11]:
class OUActionNoise:
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None):
        self.theta = theta
        self.mean = mean
        self.std_dev = std_deviation
        self.dt = dt
        self.x_initial = x_initial
        self.reset()

    def __call__(self):
        # Formula taken from https://www.wikipedia.org/wiki/Ornstein-Uhlenbeck_process
        x = (
            self.x_prev
            + self.theta * (self.mean - self.x_prev) * self.dt
            + self.std_dev * np.sqrt(self.dt) * np.random.normal(size=self.mean.shape)
        )
        # Store x into x_prev
        # Makes next noise dependent on current one
        self.x_prev = x
        return x

    def reset(self):
        if self.x_initial is not None:
            self.x_prev = self.x_initial
        else:
            self.x_prev = np.zeros_like(self.mean)

In [50]:
def policy(state, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, 2)
    action_probs = keras.backend.squeeze(actor_model(state), axis=0)
    return np.argmax(action_probs)

# This update target parameters slowly
# Based on rate `tau`, which is much less than one.
def update_target(target, original, tau):
    target_weights = target.get_weights()
    original_weights = original.get_weights()

    for i in range(len(target_weights)):
        target_weights[i] = original_weights[i] * tau + target_weights[i] * (1 - tau)

    target.set_weights(target_weights)
    
# Eager execution is turned on by default in TensorFlow 2. Decorating with tf.function allows
# TensorFlow to build a static graph out of the logic and computations in our function.
# This provides a large speed up for blocks of code that contain many small TensorFlow operations such as this one.
@tf.function
def update(
    state_batch,
    action_batch,
    reward_batch,
    next_state_batch,
):
    # Training and updating Actor & Critic networks.
    # See Pseudo Code.
    with tf.GradientTape() as tape:
        target_actions = target_actor(next_state_batch, training=True)
        y = reward_batch + gamma * target_critic(
            [next_state_batch, target_actions], training=True
        )
        critic_value = critic_model([state_batch, action_batch], training=True)
        critic_loss = keras.backend.mean(keras.backend.square(y - critic_value))

    critic_grad = tape.gradient(critic_loss, critic_model.trainable_variables)
    critic_optimizer.apply_gradients(
        zip(critic_grad, critic_model.trainable_variables)
    )

    with tf.GradientTape() as tape:
        actions = actor_model(state_batch, training=True)
        critic_value = critic_model([state_batch, actions], training=True)
        # Used `-value` as we want to maximize the value given
        # by the critic for our actions
        actor_loss = -keras.backend.mean(critic_value)

    actor_grad = tape.gradient(actor_loss, actor_model.trainable_variables)
    actor_optimizer.apply_gradients(
        zip(actor_grad, actor_model.trainable_variables)
    )

# We compute the loss and update parameters
def learn():
    if buffer.num_frames() < batch_size:
        return
    # Get sampling range
    record_range = min(buffer.num_frames(), buffer_capacity)
    # Randomly sample indices
    batch_indices = np.random.choice(record_range, batch_size)

    samples = buffer.get_next(sample_batch_size=batch_size, num_steps=1)
    samples = samples[0]
    # Convert to tensors
    state_batch = tf.squeeze(samples[0], axis=1)
    action_batch = tf.squeeze(samples[1], axis=1)
    reward_batch = tf.squeeze(samples[2], axis=1)
    reward_batch = tf.cast(reward_batch, dtype="float32")
    next_state_batch = tf.squeeze(samples[3], axis=1)

    update(state_batch, action_batch, reward_batch, next_state_batch)

In [53]:
std_dev = 1

actor_model = model.get_actor(num_states=num_states, num_actions=num_actions)
critic_model = model.get_critic(num_states=num_states, num_actions=num_actions)

target_actor = model.get_actor(num_states=num_states, num_actions=num_actions)
target_critic = model.get_critic(num_states=num_states, num_actions=num_actions)

# Making the weights equal initially
target_actor.set_weights(actor_model.get_weights())
target_critic.set_weights(critic_model.get_weights())

# Learning rate for actor-critic models
critic_lr = 0.001
actor_lr = 0.001

critic_optimizer = keras.optimizers.Adam(critic_lr)
actor_optimizer = keras.optimizers.Adam(actor_lr)

total_episodes = 100
# Discount factor for future rewards
gamma = 0.99
# Used to update target networks
tau = 0.005

batch_size = 64
buffer_capacity = 50000
buffer = rb.Buffer(num_states=num_states, num_actions=num_actions, buffer_capacity=buffer_capacity, batch_size=batch_size)

In [ ]:
# To store reward history of each episode
ep_reward_list = []
# To store average reward history of last few episodes
avg_reward_list = []

# Takes about 4 min to train
for ep in range(total_episodes):
    prev_state, _ = env.reset()
    episodic_reward = 0
    episodic_length = 0

    epsilon_initial = 1.0
    epsilon_final = 0.01
    epsilon_decay = 0.999
    epsilon = epsilon_initial

    batch_count = 0
    prev_state_batch = np.zeros((batch_size, num_states))
    action_batch = np.zeros((batch_size, num_actions))
    reward_batch = np.zeros((batch_size, 1))
    state_batch = np.zeros((batch_size, num_states))

    while True:
        tf_prev_state = prev_state[np.newaxis, :]

        epsilon = max(epsilon_final, epsilon * epsilon_decay )
        action = policy(tf_prev_state, epsilon)
        # Receive state and reward from environment.
        state, reward, done, truncated, info = env.step(action)
        reward = reward - (state[0])*0.1 #- (state[0]*state[2]-state[1]*state[3]) * 0.1
        #cos(theta1) + cos(theta1 + theta2)
        """
        #距離
        x, t = state[:1], state[1:]
        dist = math.sqrt(np.sum( (x-t)**2 ))
        pixel_size = 10
        dist = dist / pixel_size

        #壁へのめり込み
        speed = action[0][0]
        speed_minus = speed - 0.01#(dx = 0でも壁のめり込みを判定する)
        speed_plus = speed + 0.01
        angle = action[0][1]
        dx_minus = (np.cos(angle) * speed_minus, np.sin(angle) * speed_minus)
        dx_plus = (np.cos(angle) * speed_plus, np.sin(angle) * speed_plus)
        x_new_minus = x - dx_minus
        x_new_plus = x - dx_plus
        merikomi = np.sum(min(x_new_minus[0] - 0, 0) + min(x_new_minus[1] - 0, 0) + min(0, (pixel_size - 1) - x_new_plus[0]) + min(0, (pixel_size - 1) - x_new_plus[1]))

        #移動したことに対して評価
        dx = np.array([np.cos(angle) * speed, np.sin(angle) * speed], dtype = int)
        delta = np.sum(dx) * 0.1 #めり込みよりプラスにならないように
        #評価をカスタマイズ
        reward = reward - dist + merikomi + delta
        """
        prev_state_batch[batch_count] = prev_state
        action_batch[batch_count] = action
        reward_batch[batch_count] = reward
        state_batch[batch_count] = state
        batch_count += 1
        if batch_count == batch_size:
            buffer.add_batch((prev_state_batch, action_batch, reward_batch, state_batch))
            batch_count = 0

        episodic_reward += reward

        learn()

        update_target(target_actor, actor_model, tau)
        update_target(target_critic, critic_model, tau)

        # End this episode when `done` or `truncated` is True
        if done or truncated:
            
            break
        episodic_length += 1
        prev_state = state

    # Get average reward of the last 40 episodes
    avg_reward = np.mean(ep_reward_list[-40:])

    #for _ in range(80):
     #   buffer.learn()

    ep_reward_list.append(episodic_reward)

    # Mean of last 40 episodes
    avg_reward = np.mean(ep_reward_list[-40:])
    print("Episode * {} * Avg Reward is ==> {}".format(ep, avg_reward))
    avg_reward_list.append(avg_reward)

# Plotting graph
# Episodes versus Avg. Rewards
plt.plot(avg_reward_list)
plt.xlabel("Episode")
plt.ylabel("Avg. Episodic Reward")
plt.show()

Episode * 0 * Avg Reward is ==> -546.7148884058001
Episode * 1 * Avg Reward is ==> -543.3697683751584
Episode * 2 * Avg Reward is ==> -544.1035804728666


In [ ]:
# Save the weights
actor_model.save_weights("pendulum_actor.weights.h5")
critic_model.save_weights("pendulum_critic.weights.h5")

target_actor.save_weights("pendulum_target_actor.weights.h5")
target_critic.save_weights("pendulum_target_critic.weights.h5")

In [ ]:
import numpy as np

_env = gym.make(env_name, render_mode="rgb_array")
_env = gym.wrappers.RecordVideo(_env, video_folder="./", disable_logger=True)
state, _ = _env.reset()
done = False
renders = []
play_time = 0

while not done and play_time < 1000:
    play_time += 1
    state = keras.ops.expand_dims(
        keras.ops.convert_to_tensor(state), 0
    )
    action = policy(state, ou_noise)
    #print(action)
    # Receive state and reward from environment.
    state, reward, done, truncated, _ = _env.step(action[0])
_env.close()
print(f"play_time:{play_time}")

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./rl-video-episode-0.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")